In [43]:
# Transfer Stanovsky dataset
# Load all data
def select_row_data(filename):
    all_sents = open(filename, "r")
    all_data_sentences = (all_sents.read()).split("\n")
    return all_data_sentences

# Load training set
def select_dataset(sentsname, tagsname):
    train_sents = open(sentsname, "r")
    training_sentences = (train_sents.read()).split("\n")
    train_tags = open(tagsname, "r")
    training_tags = (train_tags.read()).split("\n")
    return training_sentences, training_tags

# Stanovsky dataset
all_data_sentences = select_row_data("data/train+test.oie.sents.txt")
training_sentences, training_tags = select_dataset("data/train.oie.sents.txt", "data/train.oie.tags.txt")

# Load training data (Default - Stanovsky)
def load_train_dataset(training_sentences, training_tags):
    assert len(training_sentences) == len(training_tags)
    train_s = []
    train_t = []
    for i in range(len(training_sentences)):
        if len(training_sentences[i]) > 0:
            train_s.append(training_sentences[i])
            train_t.append(training_tags[i])
        else:
            continue
    
    assert len(train_s) == len(train_t)
    training_data = [(train_s[i].split(), train_t[i].split()) for i in range(len(train_t))]
    return training_data

training_data = load_train_dataset(training_sentences, training_tags)
all_data = [all_data_sentences[i].split() for i in range(len(all_data_sentences))]
print("We have %i training data" % (len(training_data)))
print("We have %i data (training + testing)" % len(all_data))
training_data[0:5]

We have 5085 training data
We have 6813 data (training + testing)


[(['courtaulds',
   "'",
   'spinoff',
   'reflects',
   'pressure',
   'on',
   'british',
   'industry',
   'to',
   'boost',
   'share',
   'prices',
   'beyond',
   'the',
   'reach',
   'of',
   'corporate',
   'raiders'],
  ['A0-B',
   'A0-I',
   'A0-I',
   'P-B',
   'A1-B',
   'A1-I',
   'A1-I',
   'A1-I',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O']),
 (['courtaulds',
   "'",
   'spinoff',
   'reflects',
   'pressure',
   'on',
   'british',
   'industry',
   'to',
   'boost',
   'share',
   'prices',
   'beyond',
   'the',
   'reach',
   'of',
   'corporate',
   'raiders'],
  ['O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'A1-B',
   'A1-I',
   'O',
   'P-B',
   'A0-B',
   'A0-I',
   'A2-B',
   'A2-I',
   'A2-I',
   'A2-I',
   'A2-I',
   'A2-I']),
 (['the',
   'statement',
   'emphasized',
   'that',
   'holders',
   'representing',
   '75',
   '%',
   'of',
   'the',
   'shares',
   'voting',
   'at',
   'a',
   'special',
   'shareholders'

In [44]:
testing_sentences, testing_tags = select_dataset("data/test.oie.sents.txt", "data/test.oie.tags.txt")
testing_data = load_train_dataset(testing_sentences, testing_tags)
print("We have %i testing data" % (len(testing_data)))
testing_data[0:5]

with open("src-train-gs.txt", 'w') as f:
    for line in training_sentences:
        if len(line) > 0:
            f.write(line)
            f.write('\n')

We have 1724 testing data


In [45]:
def transferSeq(sents, tags):
    res = ""
    argdic = {"A0":[], "A1":[], "A2":[], "A3":[], "A4":[], "A5":[], "P-":[]}
    assert len(sents) == len(tags)
    for i in range(len(tags)):
        key = tags[i][0:2]
        if key in argdic:
            argdic[key].append(sents[i])

        if tags[i][0] == 'A':
            sub = '<arg' + tags[i][1] + '> '
            if sub not in res:
                res = res + ' ' + sub + ' ' + sents[i]
            else:
                res = res + ' ' + sents[i]
        elif tags[i][0] == 'P':
            sub = ' <rel> '
            res = res + ' ' + sub + ' ' + sents[i]
        else:
            continue
    
    arr = res.split()
    transferStr = ""
    
    i = 0
    while i < len(arr):
        if '<arg0>' in arr[i]:
            dis = len(argdic['A0'])
            temp_s = ' '.join([w for w in argdic['A0']])
            transferStr = transferStr + ' <arg0> ' + temp_s + ' </arg0> '
            i += dis
        elif '<arg1>' in arr[i]:
            dis = len(argdic['A1'])
            temp_s = ' '.join([w for w in argdic['A1']])
            transferStr = transferStr + ' <arg1> ' + temp_s + ' </arg1> '
            i += dis
        elif '<arg2>' in arr[i]:
            dis = len(argdic['A2'])
            temp_s = ' '.join([w for w in argdic['A2']])
            transferStr = transferStr + ' <arg2> ' + temp_s + ' </arg2> '
            i += dis
        elif '<arg3>' in arr[i]:
            dis = len(argdic['A3'])
            temp_s = ' '.join([w for w in argdic['A3']])
            transferStr = transferStr + ' <arg3> ' + temp_s + ' </arg3> '
            i += dis
        elif '<arg4>' in arr[i]:
            dis = len(argdic['A4'])
            temp_s = ' '.join([w for w in argdic['A4']])
            transferStr = transferStr + ' <arg4> ' + temp_s + ' </arg4> '
            i += dis
        elif '<arg5>' in arr[i]:
            dis = len(argdic['A5'])
            temp_s = ' '.join([w for w in argdic['A5']])
            transferStr = transferStr + ' <arg5> ' + temp_s + ' </arg5> '
            i += dis
        elif '<rel>' in arr[i]:
            dis = len(argdic['P-'])
            temp_s = ' '.join([w for w in argdic['P-']])
            transferStr = transferStr + ' <rel> ' + temp_s + ' </rel> '
            i += dis
        else:
            i += 1
    return transferStr

s = transferSeq(['courtaulds',"'",'spinoff','reflects','pressure','on','british','industry','to','boost','share','prices','beyond','the','reach','of','corporate','raiders'], 
                ['A0-B','A0-I','A0-I','P-B','A1-B','A1-I','A1-I','A1-I','O','O','O','O','O','O','O','O','O','O'])
s

" <arg0> courtaulds ' spinoff </arg0>  <rel> reflects </rel>  <arg1> pressure on british industry </arg1> "

In [37]:
# Get all transferred sequences
def getTripleSeq(data):    
    triples = []
    for i in range(len(data)):
        ts = transferSeq(data[i][0], data[i][1])
        triples.append(ts)
    
    return triples

def gen_train_seq(data, filename):
    tps = getTripleSeq(data)
    with open(filename, 'w') as f:
        for line in tps:
            f.write(line)
            f.write('\n')

gen_train_seq(training_data, 'tgt-src-gs-new.txt')
gen_train_seq(testing_data, 'tgt-val-gs-new.txt')

In [5]:
import torch
import torch.nn as nn

import onmt
import onmt.inputters
import onmt.modules
import onmt.utils

In [12]:
vocab_fields = torch.load("./OpenNMT-py/data/data.vocab.pt")

src_text_field = vocab_fields["src"].base_field
src_vocab = src_text_field.vocab
src_padding = src_vocab.stoi[src_text_field.pad_token]

tgt_text_field = vocab_fields["tgt"].base_field
tgt_vocab = tgt_text_field.vocab
tgt_padding = tgt_vocab.stoi[tgt_text_field.pad_token]

In [13]:
emb_size = 100
rnn_size = 500

encoder_embeddings = onmt.modules.Embeddings(emb_size, len(src_vocab), word_padding_idx = src_padding)
encoder_embeddings.load_pretrained_vectors("./OpenNMT-py/data/embeddings.enc.pt")
encoder = onmt.encoders.RNNEncoder(hidden_size = rnn_size, num_layers = 1, rnn_type = "LSTM", bidirectional = True, embeddings = encoder_embeddings)

decoder_embeddings = onmt.modules.Embeddings(emb_size, len(tgt_vocab), word_padding_idx = tgt_padding)
decoder_embeddings.load_pretrained_vectors("./OpenNMT-py/data/embeddings.dec.pt")
decoder = onmt.decoders.decoder.InputFeedRNNDecoder(hidden_size = rnn_size, num_layers = 1, bidirectional_encoder = True, rnn_type = "LSTM", embeddings = decoder_embeddings)

device = "cuda" if torch.cuda.is_available() else "cpu"
model = onmt.models.model.NMTModel(encoder, decoder)
model.to(device)

model.generator = nn.Sequential(nn.Linear(rnn_size, len(tgt_vocab)), nn.LogSoftmax(dim = 1)).to(device)

loss = onmt.utils.loss.NMTLossCompute(criterion = nn.NLLLoss(ignore_index = tgt_padding, reduction = "sum"), generator = model.generator)

In [14]:
lr = 0.01
torch_optimizer = torch.optim.SGD(model.parameters(), lr = lr)
optim = onmt.utils.optimizers.Optimizer(torch_optimizer, learning_rate = lr, max_grad_norm = 2)

In [15]:
from itertools import chain
train_data_file = "OpenNMT-py/data/data.train.0.pt"
valid_data_file = "OpenNMT-py/data/data.valid.0.pt"
train_iter = onmt.inputters.inputter.DatasetLazyIter(dataset_paths = [train_data_file], fields = vocab_fields, batch_size = 50, batch_size_multiple = 1, batch_size_fn = None, device = device, is_train = True, pool_factor = 1, repeat = True)
valid_iter = onmt.inputters.inputter.DatasetLazyIter(dataset_paths = [valid_data_file], fields = vocab_fields, batch_size = 10, batch_size_multiple = 1, batch_size_fn = None, device = device, is_train = False, pool_factor = 1, repeat = False)

In [16]:
report_manager = onmt.utils.ReportMgr(report_every = 50, start_time = None, tensorboard_writer = None)
trainer = onmt.Trainer(model = model, train_loss = loss, valid_loss = loss, optim = optim, report_manager = report_manager)
trainer.train(train_iter = train_iter, train_steps = 400, valid_iter = valid_iter, valid_steps = 200)

In [ ]:
import onmt.translate

src_reader = onmt.inputters.str2reader["text"]
tgt_reader = onmt.inputters.str2reader["text"]
scorer = onmt.translate.GNMTGlobalScorer(alpha = 0.7, beta = 0., length_penalty = "avg", coverage_penalty = "none")
gpu = 0 if torch.cuda.is_available() else -1
translator = onmt.translate.Translator(model = model, fields = vocab_fields, src_reader = src_reader, tgt_reader = tgt_reader, global_scorer = scorer, gpu = gpu)
builder = onmt.translate.TranslationBuilder(data = torch.load(valid_data_file), fields = vocab_fields)

for batch in valid_iter:
    trans_batch = translator.translate_batch(batch = batch, src_vocabs = [src_vocab], attn_debug = False)
    translations = builder.from_batch(trans_batch)
    for trans in translations:
        print(trans.log(0))

/Users/distiller/project/conda/conda-bld/pytorch_1587428061935/work/aten/src/ATen/native/BinaryOps.cpp:66: UserWarning: Integer division of tensors using div or / is deprecated, and in a future release div will perform true division as in Python 3. Use true_divide or floor_divide (// in Python) instead.



SENT 0: ['the', 'effect', 'is', 'that', 'lawsuits', 'that', 'might', 'have', 'been', 'barred', 'because', 'they', 'were', 'filed', 'too', 'late', 'could', 'proceed', 'because', 'of', 'the', 'one', '-', 'year', 'extension']
PRED 0: the the
PRED SCORE: -2.4562


SENT 0: ['the', 'effect', 'is', 'that', 'lawsuits', 'that', 'might', 'have', 'been', 'barred', 'because', 'they', 'were', 'filed', 'too', 'late', 'could', 'proceed', 'because', 'of', 'the', 'one', '-', 'year', 'extension']
PRED 0: the the
PRED SCORE: -2.4562


SENT 0: ['the', 'effect', 'is', 'that', 'lawsuits', 'that', 'might', 'have', 'been', 'barred', 'because', 'they', 'were', 'filed', 'too', 'late', 'could', 'proceed', 'because', 'of', 'the', 'one', '-', 'year', 'extension']
PRED 0: the the
PRED SCORE: -2.4562


SENT 0: ['in', '1988', ',', 'a', 'year', 'and', 'a', 'half', 'after', 'mrs.', 'marcos', 'and', 'her', 'late', 'husband', ',', 'ferdinand', 'marcos', ',', 'the', 'ousted', 'president', 'of', 'the', 'philippines', ',',


SENT 0: ['the', 'latest', '10', '-', 'year', 'notes', 'ended', 'at', 'about', '100', '16\\/32', 'to', 'yield', '7.90', '%', ',', 'compared', 'with', '100', '11\\/32', 'to', 'yield', '7.93', '%', 'on', 'friday']
PRED 0: the the
PRED SCORE: -2.5968


SENT 0: ['the', 'two', 'leaders', 'are', 'expected', 'to', 'discuss', 'changes', 'sweeping', 'the', 'east', 'bloc', 'as', 'well', 'as', 'human', '-', 'rights', 'issues', ',', 'regional', 'disputes', 'and', 'economic', 'cooperation']
PRED 0: the the
PRED SCORE: -2.4188


SENT 0: ['the', 'two', 'leaders', 'are', 'expected', 'to', 'discuss', 'changes', 'sweeping', 'the', 'east', 'bloc', 'as', 'well', 'as', 'human', '-', 'rights', 'issues', ',', 'regional', 'disputes', 'and', 'economic', 'cooperation']
PRED 0: the the
PRED SCORE: -2.4188


SENT 0: ['the', 'two', 'leaders', 'are', 'expected', 'to', 'discuss', 'changes', 'sweeping', 'the', 'east', 'bloc', 'as', 'well', 'as', 'human', '-', 'rights', 'issues', ',', 'regional', 'disputes', 'and', 'e


SENT 0: ['on', 'a', 'recent', 'afternoon', ',', 'mr.', 'baker', 'and', 'a', 'reporter', 'go', 'ghost', '-', 'busting', ',', 'visiting', 'kathleen', 'stinnett', ',', 'a', 'lexington', 'woman', 'who', 'has', 'phoned', 'the', 'university', 'of', 'kentucky', 'to', 'report', 'mysterious', 'happenings', 'in', 'her', 'house']
PRED 0: the the
PRED SCORE: -2.6059


SENT 0: ['on', 'a', 'recent', 'afternoon', ',', 'mr.', 'baker', 'and', 'a', 'reporter', 'go', 'ghost', '-', 'busting', ',', 'visiting', 'kathleen', 'stinnett', ',', 'a', 'lexington', 'woman', 'who', 'has', 'phoned', 'the', 'university', 'of', 'kentucky', 'to', 'report', 'mysterious', 'happenings', 'in', 'her', 'house']
PRED 0: the the
PRED SCORE: -2.6059


SENT 0: ['on', 'a', 'recent', 'afternoon', ',', 'mr.', 'baker', 'and', 'a', 'reporter', 'go', 'ghost', '-', 'busting', ',', 'visiting', 'kathleen', 'stinnett', ',', 'a', 'lexington', 'woman', 'who', 'has', 'phoned', 'the', 'university', 'of', 'kentucky', 'to', 'report', 'mysteriou


SENT 0: ['mr.', 'guber', 'and', 'mr.', 'peters', 'also', 'almost', 'certainly', 'would', "n't", 'be', 'able', 'to', 'participate', 'in', 'future', 'sequels', 'to', '``', 'batman', ',', "''", 'the', 'blockbuster', 'hit', 'they', 'produced', 'for', 'warner']
PRED 0: the the
PRED SCORE: -2.5377


SENT 0: ['norris', 'mclaughlin', 'is', 'a', 'general', '-', 'practice', 'firm', 'that', 'has', 'expanded', 'recently', 'into', 'such', 'specialties', 'as', 'banking', ',', 'labor', 'and', 'environmental', 'work']
PRED 0: the the
PRED SCORE: -2.6573


SENT 0: ['norris', 'mclaughlin', 'is', 'a', 'general', '-', 'practice', 'firm', 'that', 'has', 'expanded', 'recently', 'into', 'such', 'specialties', 'as', 'banking', ',', 'labor', 'and', 'environmental', 'work']
PRED 0: the the
PRED SCORE: -2.6573


SENT 0: ['the', 'conviction', 'stemmed', 'from', 'federal', 'charges', 'of', 'consumer', 'fraud', 'for', 'sale', 'of', 'phony', 'infant', 'apple', 'juice', 'between', '1978', 'and', '1983']
PRED 0: 
PRE